In this project, I'm going to explore the Kaggle baseball data set.  This file reads in the teams data from the sqlite database, does some data exploration and visualization, and finally I will do some unsupervised clustering to see if there's anything interesting there.

In [1]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Placeholder for all the shit I already did and then acidentally deleted........